In [ ]:
import initialisation
import modelTrainer
import musicPlayer
import simulation
import fitness
import music21
import collections
import importlib
import numpy as np
import pandas as pd

In [ ]:
from music21 import *

In [ ]:
importlib.reload(initialisation)
importlib.reload(modelTrainer)
importlib.reload(musicPlayer)
importlib.reload(simulation)
importlib.reload(fitness)
importlib.reload(music21)

In [ ]:
coreCorpus = music21.corpus.corpora.CoreCorpus()
sixEight = music21.corpus.search('6/8')

bachCorpusScores = []
for c in sixEight:
    score = c.parse()
    bachCorpusScores.append(score)

In [ ]:

 noteIterator = scores[0].parts[0].getElementsByClass(stream.Measure).flat.getElementsByClass('Duration')

In [ ]:
noteIterator = scores[0].getElementsByClass(stream.Measure).flat.getElementsByClass('Note').

In [ ]:
bachCorpusScores

In [ ]:
possible_durations = ['half', 'quarter', 'eighth', '16th', '32nd'] # set containing all possible notes for matrix creation
all_durations = [] # Multidimensional array of all notes per piece
testcorpus = [
                ['quarter', 'quarter', 'quarter', 'eighth', '16th'], 
                ['eighth', 'eighth', 'eighth', 'quarter', 'quarter'],
                ['16th', 'quarter', 'eighth', 'quarter', 'quarter']
             ]
last_notes = []
for i in bachCorpusScores:
    #measures = len(p.getElementsByClass(music21.stream.Part)[0].getElementsByClass(music21.stream.Measure))
    # Get a part of the piece
    score = i.parts[0].getElementsByClass(music21.stream.Measure).flat.getElementsByClass('Note')
    
    curr = []
    if(len(score) == 0):
        continue
        
    for i in range(len(score)):
        dur = score[i]

        if(i==len(score)-1):
            last_notes.append(dur.duration.type)

        if(dur.duration.type == 'zero' or dur.duration.type == 'complex' or dur.duration.type == '64th' or dur.duration.type == 'whole'):
             continue
        curr.append(dur.duration.type)
            
    all_durations.append(curr)
flatten = lambda l: [item for sublist in l for item in sublist]
last_note_counter = collections.Counter(last_notes)
counter = collections.Counter(flatten(all_durations))
counter = counter - last_note_counter

In [ ]:
counter

In [ ]:
last_note_counter

In [ ]:
zeros = np.full((len(possible_durations), len(possible_durations)), 0)
matrix = pd.DataFrame(zeros, index=possible_durations, columns=possible_durations)
matrix['half']

In [ ]:
all_durations[0]

In [ ]:
total = sum(counter.values())
zeros = np.full((len(possible_durations), len(possible_durations)), 0)
matrix = pd.DataFrame(zeros, index=possible_durations, columns=possible_durations)
matrix = matrix.astype(float)
# Fill transition matrix frequencies
for i in range(len(all_durations)):
    curr = all_durations[i]
    for j in range(len(curr)):
        # First note
        if(j == 0):
            continue

        curr_duration = curr[j-1]
        next_duration = curr[j]
        matrix[curr_duration][next_duration] = matrix[curr_duration][next_duration] + 1
matrix

In [ ]:
matrix

In [ ]:
for i in possible_durations:
    for j in possible_durations:
        print(matrix[i][j])

In [ ]:
### Divide each row to get probabilistic model
for i in possible_durations:
    for j in possible_durations:
        matrix[j][i] = matrix[j][i] / counter[j]

In [ ]:
all_durations[-1]

In [ ]:
matrix.sum(0)

In [ ]:
dur_matrix = modelTrainer.train_duration_matrix(bachCorpusScores)

In [ ]:
pitch_matrix = modelTrainer.train_pitch_matrix();

In [ ]:
population = initialisation.initialize_population(10, 8, pitch_matrix, dur_matrix)

In [ ]:
sel = population[0:3]
s2 = list(map(lambda x: x[0], sel))
s3 = list(map(lambda x: list(map(lambda y:  y[0] ,x)), s2))
s3

In [ ]:
simulation.run(2, 20, pitch_matrix, dur_matrix)